# Overview
This dataset comes from Bridges to Prosperity, an NGO

This notebook is used to extract specific columns from a paragraph of data in the `Comments` column of our dataset. The delimiters, casing, and wording sometimes varies between and within columns, so we used regular expressions for their versatility.

## Imports and reading in dataset

In [1]:
import re
import pandas as pd

In [2]:
bridges_df = pd.read_excel('B2P Dataset_2020.10.xlsx')
bridges_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 43 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Bridge Name                                      1472 non-null   object 
 1   Bridge Opportunity: Project Code                 1472 non-null   object 
 2   Bridge Opportunity: Needs Assessment             1432 non-null   object 
 3   Bridge Opportunity: Level 1 Government           1472 non-null   object 
 4   Bridge Opportunity: Level 2 Government           1472 non-null   object 
 5   Bridge Opportunity: Stage                        1472 non-null   object 
 6   Bridge Opportunity: GPS (Latitude)               1460 non-null   float64
 7   Bridge Opportunity: GPS (Longitude)              1460 non-null   float64
 8   Bridge Opportunity: Bridge Type                  185 non-null    object 
 9   Bridge Opportunity: Span (m)  

In [3]:
# Display all columns, without truncation
pd.set_option('display.max_columns', None)
bridges_df

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Bridge Opportunity: Comments,Form: Form Name,Form: Created By,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Incident descriptions,Notes on social information,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: General Project Photos,Bridge Opportunity: CaseSafeID,Senior Engineering Review Conducted
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,Cancelled,-2.760833,29.488056,Suspended Bridge,NaN,NaN,Steep slopes to deal with for anchors,Project Assessment - 2018.10.10,aimablengirabakunzi taroworks,-2.760833,29.488056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huye,NaN,NaN,Yes,There is a planned vehicular road with a vehic...,NaN,NaN,NaN,NaN,006f100000asFErAAM,Yes
1,Kagarama,1007325,NaN,Western Province,Nyamasheke,Complete,-2.322534,29.141944,Suspension Bridge,48.0,4000.0,NaN,Project Assessment - 2017.7.12,Stephanie May,-2.322534,29.141944,timber bridge,2km downstream,120.0,12hours,"Schools, hospital, markets and government serv...",NaN,NaN,NaN,farming,banana,2.0,15.0,the injuries and deaths are caused by trying f...,Most of the facilities locate at the left side...,Fair,Seasonally,Kibogora-Kabuga,Karongi-Nyamasheke,NaN,NaN,NaN,NaN,10.0,0.7,https://flic.kr/s/aHskvedfPB,006f100000a82QzAAI,Yes
2,Karambi,1007374,NaN,Southern Province,Huye,Complete,-2.513333,29.612778,Suspension Bridge,50.0,12500.0,Likely piloting steel decking on this bridge. ...,Project Assessment - 2018.7.27,etiennemutebutsi taroworks,NaN,NaN,Timber bridge,Is located at 5km upstream. People pay 1000rwf...,60.0,1day,"Markets,farms blocked by this river both sides",Education blocked by this river both sides,Health services blocked by this river both sides,"Potable water, places of worship ,places for r...",Farming and small business,"Corns, banana and beans",3.0,15.0,"Peoples dead when coming from the church, heal...","5 cells (kabuye,gasumba,kanyinya, buremera and...",Good,Never,Nyamagabe,Huye-Nyamagabe,Standard,NaN,NaN,Suspension,45.0,1.0,https://flic.kr/s/aHskbMF1Kr,006f100000a86DaAAI,Yes
3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.476407,29.420800,Suspension Bridge,68.0,4363.0,"3000-6000 people directly served, Elevation:21...",Project Assessment - 2018.9.1,aimablengirabakunzi taroworks,-2.476407,29.420800,Timber,None,60.0,All time in April and May and it is rarely to ...,Markets,Primary and Secondary schools,Health Center,Churches and Government offices,"Farming of large tea plantation, wheat farms","Tea, cow pea, wheat, beans, sweet potatoes",0.0,2.0,They slipped on trees and fall down in the river,"Market 1( Minaga, Gishwati, Mujerenge, Gikungu...",Fair,Seasonally,Huye,Nyamagabe-Rusizi,Standard,No,NaN,Suspended,47.4,1.1,https://www.flickr.com/photos/bridgestoprosper...,006f100000a86F4AAI,Yes
4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.233022,29.575130,Suspension Bridge,78.0,2790.0,"Not Duplicate 500-1000 people directly served,...",Project Assessment - 2018.9.4,dominique

## Test Regex Matches

In [4]:
bridges_df['Bridge Opportunity: Comments'].value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [5]:
bridges_df['Bridge Opportunity: Comments'][2]

'Likely piloting steel decking on this bridge. 6000-10000 people directly served, Elevation:1630m, Cell:Karambi-Gasumba, Injuries/Death-many peoples injured while trying to cross the river/ 2 peoples died while trying to cross the river Cross river on a normal day->600 people, Nearby city centers--Karambi -Nyamagabe, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-6-9 months / Hagati y?amezi atandatu n?icyenda,Travel to nearest safe bridge/river crossing->3 hours / Hejuru y?amasaha 3,Hours walking to reach the Hospital-0.5-1 hours,Hours walking to reach the Health Center-0.5 hours,Hours of walking to reach the market-0.5 hours,Hours walking to reach Primary School-0.5 hours,Hours walking to reach Secondary School-0.5 hours,Hours walking to reach the Church-0.5 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka 

In [6]:
# Example of paragraph of data
test_string_2 = bridges_df['Bridge Opportunity: Comments'][2]
test_string_2

'Likely piloting steel decking on this bridge. 6000-10000 people directly served, Elevation:1630m, Cell:Karambi-Gasumba, Injuries/Death-many peoples injured while trying to cross the river/ 2 peoples died while trying to cross the river Cross river on a normal day->600 people, Nearby city centers--Karambi -Nyamagabe, Crossing River now-Simple timber bridge / hari uduti, Impossible/Dangerous to cross the river-6-9 months / Hagati y?amezi atandatu n?icyenda,Travel to nearest safe bridge/river crossing->3 hours / Hejuru y?amasaha 3,Hours walking to reach the Hospital-0.5-1 hours,Hours walking to reach the Health Center-0.5 hours,Hours of walking to reach the market-0.5 hours,Hours walking to reach Primary School-0.5 hours,Hours walking to reach Secondary School-0.5 hours,Hours walking to reach the Church-0.5 hours,Land within 50m of river bank-flat (flood plain) / ikibaya,Soil-Silt,Sand-Available / birahaboneka,Gravel-Not available on site, but local government can provide/ Ntibihaboneka 

In [7]:
# Matches range of people directly served, e.g. 6000-10000
# Case insensitive (?i)
people_directly_served = re.search(
    pattern=r'(?i)([\d-]+) people directly served', 
    string=test_string_2
)

if people_directly_served: print(people_directly_served.group(1))

6000-10000


In [8]:
# Matches elevation in meters
# Note: Change column name to be Elevation (meters) so each column value can be an integer
# Then cast this string as int
elevation = re.search(
    pattern=r'(?i)Elevation:(\d+)', 
    string=test_string_2
)

if elevation: print(elevation.group(1))

1630


In [9]:
# Matches the 'cell' of the bridge site, optional space after Cell, optional dash between
# word characters
cell = re.search(
    pattern=r'(?i)Cell: ?(\w+\-?\w*)',
    string=test_string_2
)

if cell: print(cell.group(1))

Karambi-Gasumba


In [10]:
bridge_connection = re.search(
    pattern=r'(?i)Bridge(?:[\s\w]+)?connect(?:s|ing)? ([\w\s\-&]+)', 
    string=test_string_2
)
if bridge_connection: print(bridge_connection.group(1)) # None

In [11]:
injuries = re.search(
    pattern=r'([\w\s]+) injur', 
    string=test_string_2
)
if injuries: print(injuries.group(1))

many peoples


In [12]:
deaths = re.search(
    pattern=r'([\w\s]+) die', 
    string=test_string_2
)
if deaths: print(deaths.group(1))

 2 peoples


In [13]:
crossing_frequency = re.search(
    pattern=r'(?i)Cross river on a normal day-?([>\d-]+)', 
    string=test_string_2
)

if crossing_frequency: print(crossing_frequency.group(1))

>600


In [14]:
nearby_city_centers = re.search(
    pattern=r'Nearby city centers--?([a-zA-Z -]*)', 
    string=test_string_2
)

if nearby_city_centers: print(nearby_city_centers.group(1).replace(' -', ', '))

Karambi, Nyamagabe


In [15]:
current_crossing_method = re.search(
    pattern=r'(?i)Crossing River now-([\w\s]+)',
    string=test_string_2
)
if current_crossing_method: print(current_crossing_method.group(1))

Simple timber bridge 


In [16]:
river_crossing_difficulty = re.search(
    pattern=r'(?i)Impossible/Dangerous to cross the river-([\w\s\-?]+)', 
    string=test_string_2
)

if river_crossing_difficulty: print(river_crossing_difficulty.group(1))

6-9 months 


In [17]:
hours_to_nearest_crossing = re.search(
    pattern=r'(?i)Travel to nearest safe bridge/river crossing-([>\w\s]+)', 
    string=test_string_2)

if hours_to_nearest_crossing: print(hours_to_nearest_crossing.group(1))

>3 hours 


In [18]:
hours_to_hospital = re.search(
    pattern=r'(?i)Hours walking to reach the Hospital-([.\d-]+)', 
    string=test_string_2
)

if hours_to_hospital: print(hours_to_hospital.group(1))

0.5-1


In [19]:
hours_to_health_center = re.search(
    pattern=r'(?i)Hours walking to reach the Health Center-([.\d-]+)', 
    string=test_string_2
)

if hours_to_health_center: print(hours_to_health_center.group(1))

0.5


In [20]:
# Rename to Hours walking (no of), like the rest of the columns
hours_to_market = re.search(
    pattern=r'(?i)Hours of walking to reach the market-([.\d-]+)', 
    string=test_string_2
)

if hours_to_market: print(hours_to_market.group(1))

0.5


In [21]:
hours_to_primary_school = re.search(
    pattern=r'(?i)Hours walking to reach Primary School-([.\d-]+)', 
    string=test_string_2
)

if hours_to_primary_school: print(hours_to_primary_school.group(1))

0.5


In [22]:
hours_to_secondary_school = re.search(
    pattern=r'(?i)Hours walking to reach Secondary School-([.\d-]+)', 
    string=test_string_2)

if hours_to_secondary_school: print(hours_to_secondary_school.group(1))

0.5


In [23]:
hours_to_church = re.search(
    pattern=r'(?i)Hours walking to reach the Church-([.\d-]+)', 
    string=test_string_2
)

if hours_to_church: print(hours_to_church.group(1))

0.5


In [24]:
land_by_river_bank = re.search(
    pattern=r'(?i)Land within 50m of river bank-([\w\s()]+)', 
    string=test_string_2
)

if land_by_river_bank: print(land_by_river_bank.group(1))

flat (flood plain) 


In [25]:
# Sometimes next column is separated with comma
soil = re.search(
    pattern=r'Soil-([\w\s]+),Sand', 
    string=test_string_2
)

if soil: print(soil.group(1))

Silt


In [26]:
sand_availability = re.search(
    pattern=r'Sand-([\w\s]+)', 
    string=test_string_2
)

if sand_availability: print(sand_availability.group(1))

Available 


In [27]:
gravel_availability = re.search(
    pattern=r'Gravel-([\w\s,]+)', 
    string=test_string_2
)

if gravel_availability: print(gravel_availability.group(1))

Not available on site, but local government can provide


In [28]:
stone_availability = re.search(
    pattern=r'Stone-([\w\s,]+) ?/', 
    string=test_string_2
)

if stone_availability: print(stone_availability.group(1))

Available 


In [29]:
timber_availability = re.search(
    pattern=r'Timber-([\w\s,]+)', 
    string=test_string_2
)

if timber_availability: print(timber_availability.group(1))

Available 


In [30]:
stone_provided_by = re.search(
    pattern=r'Stone provided by-([\w\s,]+)', 
    string=test_string_2
)

if stone_provided_by: print(stone_provided_by.group(1))

Sector


In [31]:
sand_provided_by = re.search(
    pattern=r'Sand Provided by-([\w\s,]+)', 
    string=test_string_2
)

if sand_provided_by: print(sand_provided_by.group(1))

Sector


In [32]:
gravel_provided_by = re.search(
    pattern=r'Gravel provided by-([\w\s,]+)', 
    string=test_string_2
)

if gravel_provided_by: print(gravel_provided_by.group(1))

District


In [33]:
timber_provided_by = re.search(
    pattern=r'Timber provided by-([\w\s,]+)', 
    string=test_string_2
)

if timber_provided_by: print(timber_provided_by.group(1))

District


In [34]:
cement_provided_by = re.search(
    pattern=r'Cement provided by-([\w\s,]+)', 
    string=test_string_2
)

if cement_provided_by: print(cement_provided_by.group(1))

Can not be provided at the time 


In [35]:
reinforcement_steel_provided_by = re.search(
    pattern=r'Reinforcement steel provided by-([\w\s,]+)', 
    string=test_string_2
)

if reinforcement_steel_provided_by: print(reinforcement_steel_provided_by.group(1))

Can not be provided at the time 


In [36]:
land_ownership = re.search(
    pattern=r'Land ownership-([\w\s,]+)', 
    string=test_string_2
)

if land_ownership: print(land_ownership.group(1))

Private landowner


In [37]:
land_ownership_permission = re.search(
    pattern=r'Land ownership permission-([\w\s,]+)', 
    string=test_string_2
)

if land_ownership_permission: print(land_ownership_permission.group(1))

Yes 


In [38]:
proposed_bridge_location = re.search(
    pattern=r'proposed bridge location is ([\w\s,]+).?-', 
    string=test_string_2
)
if proposed_bridge_location: print(proposed_bridge_location.group(1))

near the traditional crossing point at a distance of approximately 10m


In [39]:
# Strip m from values, put m in column name
proposed_bridge_span = re.search(
    pattern=r'proposed bridge span is (?:approximately )?(\d+)\w*.?-', 
    string=test_string_2
)

if proposed_bridge_span: print(proposed_bridge_span.group(1))

70


In [40]:
# Also in meters
level_difference_between_two_banks = re.search(
    pattern=r'level difference between two banks is ([\d.]+)', 
    string=test_string_2
)

if level_difference_between_two_banks: print(level_difference_between_two_banks.group(1))

0.06


In [41]:
space_for_foundation = re.search(
    pattern=r'space for foundation is (\w+\s*)', 
    string=test_string_2
)
if space_for_foundation: print(space_for_foundation.group(1))

sufficient


In [42]:
free_board = re.search(
    pattern=r'free board between the lowest point of the proposed bridge and the highest flood level(?: is)? (\w+\s*)',
    string=test_string_2
)
if free_board: print(free_board.group(1))

sufficient


In [43]:
river_bed_status = re.search(
    pattern=r'river bed at the site is ([\w\s,]+)', 
    string=test_string_2
)
if river_bed_status: print(river_bed_status.group(1))

stable, there is no possibility of erosion


In [44]:
river_bank_status = re.search(
    r'river bank of the site is ([\w\s,]+)', 
    string=test_string_2
)
if river_bank_status: print(river_bank_status.group(1))

not showing any sign of erosion and it is located between flat areas


In [45]:
soil_from_site = re.search(
    pattern=r'soil from the site is ([\w\s]+)', 
    string=test_string_2
)

if soil_from_site: print(soil_from_site.group(1))

silt for both sides


In [46]:
confluence_area_near_place = re.search(
    r'([\w\s]+\bconfluence\b[\w\s]+)', 
    string=test_string_2
)

if confluence_area_near_place: print(confluence_area_near_place.group(1))

There is no confluence area near the place


## Editing Features
Now that we've found regular expressions to match data in the comments column, we will add them to the dataset by either engineering new features or editing existing ones. Since the comments data is from 2013 while the other column data is from 2018, if data already exists in a separate column, we will keep it. We will only add old data to an existing column if that column's row is null.

### Columns in comments data that correspond to existing columns
1. `People directly served` (corresponds to `Bridge Opportunity: Individuals Directly Served`)
2. `Injuries` (roughly corresponds to `River crossing injuries in last 3 years`)
3. `Deaths` (roughly corresponds to `River crossing deaths in last 3 years`)
4. `Nearby city centers` (corresponds to `Name of nearest city`)
5. `Crossing River now` (corresponds to `Current crossing method`)

First, we'll map extracted data to their corresponding 5 existing columns wherever the values in those columns are null.

In [47]:
comments = bridges_df['Bridge Opportunity: Comments']
extracted = bridges_df.copy()

# Boolean array used to check whether there is data to extract
non_nan_comments = bridges_df['Bridge Opportunity: Comments'].notna()

# Boolean arrays used to check if values in existing columns are null (in which case
# they will be changed, else they will be left the same)
individuals_served_nans = bridges_df['Bridge Opportunity: Individuals Directly Served'].notna()
injuries_nans = bridges_df['River crossing injuries in last 3 years'].notna()
deaths_nans = bridges_df['River crossing deaths in last 3 years'].notna()
nearest_city_nans = bridges_df['Name of nearest city'].notna()
river_crossing_nans = bridges_df['Current crossing method'].notna()

In [48]:
comment_count = 0
for i in range(len(bridges_df)):
    if non_nan_comments[i]:
        if individuals_served_nans[i]:
            people_directly_served = re.search(
                pattern=r'(?i)([\d-]+) people directly served', 
                string=comments[i])
            if people_directly_served:
                # This is a range separated by -. Take its midpoint.
                nums = [int(num) for num in people_directly_served.group(1).split('-')]
                avg = sum(nums) / len(nums)
                extracted['Bridge Opportunity: Individuals Directly Served'][i] = avg

        if injuries_nans[i]:
            injuries = re.search(pattern=r'([\w\s]+) injur', string=comments[i])
            if injuries:
                extracted['River crossing injuries in last 3 years'][i] = injuries.group(1)
            
        if deaths_nans[i]:
            deaths = re.search(pattern=r'([\w\s]+) die', string=comments[i])
            if deaths:
                extracted['River crossing deaths in last 3 years'][i] = deaths.group(1)
        
        if nearest_city_nans[i]:
            nearest_city = re.search(
                pattern=r'Nearby city centers--?([a-zA-Z -]*)', 
                string=comments[i])
            if nearest_city:
                extracted['Name of nearest city'][i] = nearest_city.group(1).replace(' -', ', ')
        
        if river_crossing_nans[i]:
            river_crossing = re.search(
                pattern=r'(?i)Crossing River now-([\w\s]+)',
                string=comments[i])
            if river_crossing:
                extracted['Current crossing method'][i] = river_crossing.group(1)
        comment_count += 1

/Users/RyanKoul/.local/share/virtualenvs/labs1-W26pRego/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/RyanKoul/.local/share/virtualenvs/labs1-W26pRego/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/RyanKoul/.local/share/virtualenvs/labs1-W26pRego/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [49]:
assert comment_count == sum(bridges_df['Bridge Opportunity: Comments'].notna())

In [ ]:
df.to_csv('b2p_cleaned.csv')
from google.colab import files
files.download('b2p_cleaned.csv')

## Engineering Features
Now, we'll make new columns for the rest of the values extracted from the comments column.

In [ ]:
new_columns = ['Elevation', 'Cell', 'Average Number of Daily Crossings', 
              'How long is it impossible/dangerous to cross river',
              'Hours to nearest safe bridge/river crossing', 'Hours walking to Hospital',
              'Hours walking to Health Center', 'Hours walking to market',
              'Hours walking to Primary School']